In [49]:
# read data from mongodb
from pymongo import MongoClient
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession

In [50]:
# connect to mongodb in localhost and access database and collections
def _connect_mongo():
    """ A util for making a connection to mongo """
    host = "localhost"
    port = 27017
    username = ""
    password = ""
    db = "5gopt"

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)


    return conn[db]


In [51]:
# get specific colelction, and remove the "_id" collumn
def collection_read_mongo(collection, query={}, no_id = True):
    db = _connect_mongo()
    cursor = db[collection].find(query)
    df = pd.DataFrame(list(cursor))

    if no_id:
        try:
            del df["_id"]
        except:
            pass
    return df

In [52]:
citiesName = ["Amsterdam", "Athens", "Barcelona", "Berlin", "Helsinki", "Paris", "Stockholm", 
              "London", "Manchester", "Roma", "Prague", "Edinburgh", "Vienna", "Lisbon", "Budapest",
             "Madrid", "Warsaw", "Brussels", "Dublin", "Munich", "Riga", "hamburg",
             "Vilnius", "Tallinn", "Reykjavik", "Frankfurt", "Zurich", 
              "liverpool", "Minsk", "Oslo", "Kiev", "Bucharest", "Sofia", "Cologne", "Kharkiv"]

In [53]:
len(citiesName)

35

In [54]:
"Roma" in citiesName

True

In [55]:
# convert cities name into all-lowercase name, and then check whether there are any duplicating names
citiesName = [i.lower() for i in citiesName]
duplicate = [i for i in citiesName if citiesName.count(i) > 1]
if len(duplicate) > 1:
    print("the list of cities name doesn't contain all cities, CHECK, CHECK")

citiesName = set(citiesName)

In [56]:
citiesName

{'amsterdam',
 'athens',
 'barcelona',
 'berlin',
 'brussels',
 'bucharest',
 'budapest',
 'cologne',
 'dublin',
 'edinburgh',
 'frankfurt',
 'hamburg',
 'helsinki',
 'kharkiv',
 'kiev',
 'lisbon',
 'liverpool',
 'london',
 'madrid',
 'manchester',
 'minsk',
 'munich',
 'oslo',
 'paris',
 'prague',
 'reykjavik',
 'riga',
 'roma',
 'sofia',
 'stockholm',
 'tallinn',
 'vienna',
 'vilnius',
 'warsaw',
 'zurich'}

In [57]:
# iterately access each mongodb collection and compare their data size, then rank them in order
client = MongoClient()

In [58]:
db = client.test

In [59]:
def changeCollectionName():
    '''
    this function only works for changing mongo collection name
    '''
    collNames = db.collection_names()
    for i in collNames:
        subNameList = i.split("_")
        if len(subNameList) > 1:
            if subNameList[1] in ["tripadvisor", "Tripadvisor"]:
#                 print("True")
                newName = subNameList[0].lower() + "Tripadvisor"
            if subNameList[1] in ["Booking", "booking"]:
                newName = subNameList[0].lower() + "Booking"
            else:#                 using original nameri
                newName = "_".join(subNameList)
            db[i].rename(newName)
        else:
            pass
    return db.collection_names()

In [60]:
print(db.collection_names())
tripCollNames = list()
bookingCollNames = list()

['hamburgBooking', 'amsterdam3Tripadvisor', 'manchesterTripadvisor', 'liverpoolBooking', 'frankfurtBooking', 'athensTripadvisor', 'bucharestBooking', 'lisbonBooking', 'romeBooking', 'osloBooking', 'helsinkiBooking', 'london2Tripadvisor', 'paris2Tripadvisor', 'viennaBooking', 'madridBooking', 'rigaBooking', 'parisTripadvisor', 'brusselsBooking', 'reykjavikBooking', 'copenhagenBooking', 'athensBooking', 'barcelonaTripadvisor', 'munichBooking', 'amsterdam2Tripadvisor', 'stockholmBooking', 'copenhagenTripadvisor', 'berlinBooking', 'dublinBooking', 'dublinTripadvisor', 'helsinkiTripadvisor', 'frankfurtTripadvisor', 'berlinTripadvisor', 'warsawBooking', 'minskBooking', 'stockholmTripadvisor', 'pragueBooking', 'tallinnBooking', 'kharkivBooking', 'londonTripadvisor', 'munichTripadvisor', 'budapestBooking', 'brusselTripadvisor', 'parisBooking', 'edinburghBooking', 'liverpoolTripadvisor', 'edinburghTripadvisor', 'kievBooking', 'barcelonaBooking', 'olsoTripadvisor', 'amsterdamTripadvisor', 'vilni

In [61]:
- except:
            size_in_booking = None
            countNum_in_booking = None
#             pass
        dataSizeList.append([i, size_in_trip, countNum_in_trip, size_in_booking, countNum_in_booking])

dataSizeDF = pd.DataFrame(dataSizeList, columns=["CityName", "Tripadvisor(MB)","CountInTrip", "Booking(MB)", "CountInBooking"])
    
    

In [62]:
dataSizeDF

,CityName,Tripadvisor(MB),CountInTrip,Booking(MB),CountInBooking
0,roma,NaN,NaN,NaN,NaN
1,helsinki,24.370316,23410.0,15.736367,23572.0
2,barcelona,161.262398,143125.0,114.565245,172146.0
3,cologne,NaN,NaN,23.938894,36514.0
4,riga,NaN,NaN,19.211486,30145.0
5,paris,504.663424,505681.0,86.797755,132728.0
6,minsk,NaN,NaN,2.666970,4059.0
7,kiev,NaN,NaN,13.847537,21423.0
8,amsterdam,224.306333,193163.0,NaN,NaN
9,madrid,NaN,NaN,70.367912,108125.0


In [63]:
# sort dataframe only according to column "Booking.com"
top10Trpadvisor = dataSizeDF.sort_values(by="Tripadvisor(MB)", ascending=False)[:10][["CityName", "Tripadvisor(MB)", "CountInTrip"]]
top10Trpadvisor

,CityName,Tripadvisor(MB),CountInTrip
5,paris,504.663424,505681.0
8,amsterdam,224.306333,193163.0
29,dublin,174.130999,159551.0
2,barcelona,161.262398,143125.0
22,london,106.319992,94869.0
21,athens,86.814761,79790.0
1,helsinki,24.370316,23410.0
33,munich,20.163715,17912.0
32,frankfurt,19.770403,17863.0
30,reykjavik,9.837871,7970.0


In [64]:
# sort dataframe only according to column "Booking.com"
top10Booking = dataSizeDF.sort_values(by="Booking(MB)", ascending=False)[:10][["CityName", "Booking(MB)", "CountInBooking"]]
top10Booking

,CityName,Booking(MB),CountInBooking
26,edinburgh,128.539648,192517.0
2,barcelona,114.565245,172146.0
29,dublin,111.184878,173875.0
15,berlin,102.383070,151977.0
27,manchester,102.170732,152434.0
5,paris,86.797755,132728.0
25,liverpool,86.559738,134681.0
9,madrid,70.367912,108125.0
12,lisbon,61.561478,89450.0
21,athens,59.329007,91615.0


In [65]:
# from pyspark.sql import SparkSession

# spark = SparkSession.builder.appName("Python Spark SQL basic example").getOrCreate()

# logger = spark._jvm.org.apache.log4j
# logger.LogManager.getRootLogger().setLevel(logger.Level.FATAL)

# # Save some data
# characters = spark.createDataFrame([("Bilbo Baggins",  50), ("Gandalf", 1000), ("Thorin", 195), ("Balin", 178), ("Kili", 77), ("Dwalin", 169), ("Oin", 167), ("Gloin", 158), ("Fili", 82), ("Bombur", None)], ["name", "age"])
# characters.write.format("com.mongodb.spark.sql").mode("overwrite").save()

# # print the schema
# print("Schema:")
# characters.printSchema()

# # read from MongoDB collection
# df = spark.read.format("com.mongodb.spark.sql").load()

# # SQL
# df.registerTempTable("temp")
# centenarians = spark.sql("SELECT name, age FROM temp WHERE age >= 100")
# print("Centenarians:")
# centenarians.show()

In [66]:
import json
from odo import odo

In [67]:
testCollTripadvisorNames = [i+"Tripadvisor" for i in citiesName]
testCollBookingNames = [j+"Booking" for j in citiesName]

In [68]:
# testCollTripadvisorNames

In [69]:
#do statistics about review count.


testCollNames = testCollTripadvisorNames + testCollBookingNames

newDatabaseName = "sentimentAnalysis"
newDB = client[newDatabaseName]
# sentAnalysisList = list()
for i in testCollNames:
#     print(type(db[i]))
#     fetch collection data from mongo, and then calcualte the statistics, we can do it just within mongo, which
# will be more efficient, but we also need these data stored in new dataset, sentimentAnalysis. So we should 
# get collections first.
#     for some cities, there is no review data in booking.com or tripadvisor.com
    try:
        cursor = db[i].find({})
        df = pd.DataFrame(list(cursor))
    #     dataDF = pd.DataFrame(db[i].find({}))
    #     print("the original dataframe shape:    ", df.shape)
        df.drop_duplicates(inplace=True)
    #     print("the dataframe shape after removing duplication:    ", df.shape)
        df.dropna(axis=0, how="any", inplace=True)
    #     print("the dataframe shape after removing any null column values:    ", df.shape)
        newDB.create_collection(i)
        odo(df, newDB[i])

    #     records = json.loads(df.T.to_json()).values()

    #     newDB[i].insert_many(records)
#         sentAnalysisList.append([i ,df.shape[0]])
    except:
        pass
#     newDB = client.newDatabaseName
#     newDB[i]
    
#     print(df[:1])
#     print(df[:1]["date"])
#     print(type(df))

# sentAnalysisDF = pd.DataFrame(sentAnalysisList, columns=["CollName", "ReviewCount"])

# remove review records which contain absent value

# print sample json data in booking json raw data

# sample json data in tripadvisor

In [70]:
sentAnalysisDF

,CollName,ReviewCount
0,helsinkiTripadvisor,450
1,barcelonaTripadvisor,3072
2,amsterdamTripadvisor,2488
3,berlinTripadvisor,941
4,stockholmTripadvisor,305
5,athensTripadvisor,1703
6,dublinTripadvisor,2840
7,reykjavikTripadvisor,103
8,frankfurtTripadvisor,85
9,munichTripadvisor,34


/home/yi/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/yi/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [73]:
sentBookingDF[:10]

,CollName,ReviewCount
33,edinburghBooking,139707
36,dublinBooking,137109
34,manchesterBooking,121142
11,barcelonaBooking,113820
23,berlinBooking,112279
14,parisBooking,111383
32,liverpoolBooking,97252
17,madridBooking,79523
29,athensBooking,70678
40,munichBooking,69763


In [74]:
sentTripadvisorDF[:10]

,CollName,ReviewCount
1,barcelonaTripadvisor,3072
6,dublinTripadvisor,2840
2,amsterdamTripadvisor,2488
5,athensTripadvisor,1703
3,berlinTripadvisor,941
0,helsinkiTripadvisor,450
4,stockholmTripadvisor,305
7,reykjavikTripadvisor,103
8,frankfurtTripadvisor,85
9,munichTripadvisor,34


In [78]:
sentBookingDF

,CollName,ReviewCount
33,edinburghBooking,139707
36,dublinBooking,137109
34,manchesterBooking,121142
11,barcelonaBooking,113820
23,berlinBooking,112279
14,parisBooking,111383
32,liverpoolBooking,97252
17,madridBooking,79523
29,athensBooking,70678
40,munichBooking,69763


In [80]:
# get each collection data size for both booking.com and tripadvisor.com
def collection_statistics(dbName=""):
    tripadvisorSuffix = "Tripadvisor"
    bookingSuffix = "Booking"
    collsSizeList = list()
    ByteToMB = 1024*1024
#     get the specific database
    db = client[dbName]
    for i in citiesName:
        if i == "":
            pass
        else:
    #         print(citiesName)
    #         print(i)
            collTripadvisor = i+tripadvisorSuffix
            collBooking = i+bookingSuffix
        #     get the collection size and then convet to MB from bytes
        #     print(collTripadvisor)
    #     since for booking or tripadvisor, some cities data doesn't be collected.
            try:
                size_in_trip = db.command("collstats", collTripadvisor)["size"] / ByteToMB
                countNum_in_trip = db.command("collstats", collTripadvisor)["count"]
            except:
                size_in_trip = None
                countNum_in_trip = None
    #             pass
            try:
                size_in_booking = db.command("collstats", collBooking)["size"] / ByteToMB
                countNum_in_booking = db.command("collstats", collBooking)["count"]
            except:
                size_in_booking = None
                countNum_in_booking = None
    #             pass
            collsSizeList.append([i, size_in_trip, countNum_in_trip, size_in_booking, countNum_in_booking])

    collsSizeDF = pd.DataFrame(collsSizeList, columns=["CityName", "Tripadvisor(MB)","CountInTrip", "Booking(MB)", "CountInBooking"])
    return collsSizeDF

In [83]:
sentCollStatDF = collection_statistics(dbName="sentimentAnalysis")


In [84]:
sentCollStatDF

,CityName,Tripadvisor(MB),CountInTrip,Booking(MB),CountInBooking
0,roma,NaN,NaN,NaN,NaN
1,helsinki,0.430638,450.0,12.135236,17911.0
2,barcelona,3.296254,3072.0,74.956341,113820.0
3,cologne,NaN,NaN,18.116894,27467.0
4,riga,NaN,NaN,11.997529,18850.0
5,paris,0.000000,0.0,73.127062,111383.0
6,minsk,NaN,NaN,1.416744,2154.0
7,kiev,NaN,NaN,6.675123,10541.0
8,amsterdam,2.754058,2488.0,NaN,NaN
9,madrid,NaN,NaN,51.602329,79523.0


In [86]:
# get "Booking" new collecton statistics seperated from the whole dataframe
sentBookingDF = sentCollStatDF.sort_values(by="CountInTrip", ascending=False)[["CityName", "Tripadvisor(MB)", "CountInTrip"]]
# get "Tripadvisor" new collecton statistics seperated from the whole dataframe
sentTripadvisorDF = sentCollStatDF.sort_values(by="CountInBooking", ascending=False)[["CityName", "Booking(MB)", "CountInBooking"]]


In [88]:
sentBookingDF[:10]

,CityName,Tripadvisor(MB),CountInTrip
2,barcelona,3.296254,3072.0
29,dublin,2.921613,2840.0
8,amsterdam,2.754058,2488.0
21,athens,1.782245,1703.0
15,berlin,0.974744,941.0
1,helsinki,0.430638,450.0
18,stockholm,0.307060,305.0
30,reykjavik,0.120811,103.0
32,frankfurt,0.091442,85.0
33,munich,0.044818,34.0


In [89]:
sentTripadvisorDF[:10]

,CityName,Booking(MB),CountInBooking
26,edinburgh,93.924212,139707.0
29,dublin,88.180438,137109.0
27,manchester,81.959416,121142.0
2,barcelona,74.956341,113820.0
15,berlin,76.382092,112279.0
5,paris,73.127062,111383.0
25,liverpool,63.423845,97252.0
9,madrid,51.602329,79523.0
21,athens,45.858156,70678.0
33,munich,45.406182,69763.0


In [98]:
commonCities = set(sentBookingDF["CityName"][:10]) - (set(sentBookingDF["CityName"][:10]) - set(sentTripadvisorDF["CityName"][:10]))

In [99]:
commonCities

{'athens', 'barcelona', 'berlin', 'dublin', 'munich'}